## LLM Agents

* Local implementation following this tutorial: https://graphacademy.neo4j.com/courses/llm-fundamentals/3-intro-to-langchain/4-agents/

### Requirements

In [10]:
!pip install langchain openai langchain-openai python-dotenv langchainhub --quiet

In [12]:
%load_ext watermark
%watermark -p langchain,langchainhub

The watermark extension is already loaded. To reload it, use:
  %reload_ext watermark
langchain   : 0.1.4
langchainhub: 0.1.14



### Imports

In [122]:
import os
from graphdatascience import GraphDataScience
from dotenv import load_dotenv, find_dotenv
from pathlib import Path

### Settings

In [123]:
project_path = Path(os.getcwd()).parent
data_path = project_path / "data"
model_path = project_path / "models"
output_path = project_path / "output"

# load env settings
_ = load_dotenv(find_dotenv())

llm_model = "gpt-4"
database = "recommendations-50"

openai_api_key = os.getenv('OPENAI_API_KEY')

#### Movie trailer agent

In [124]:
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains.conversation.memory import ConversationBufferMemory
from langchain.agents import AgentExecutor, create_react_agent
from langchain.tools import Tool
from langchain import hub

llm = ChatOpenAI(openai_api_key=openai_api_key)

prompt = PromptTemplate(
    template="""
    You are a movie expert. You find movies from a genre or plot.

    ChatHistory:{chat_history}
    Question:{input}
    """,
    input_variables=["chat_history", "input"],
)

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

chat_chain = LLMChain(llm=llm, prompt=prompt, memory=memory, verbose=True)

tools = [
    Tool.from_function(
        name="Movie Chat",
        description="For when you need to chat about movies. The question will be a string. Return a string.",
        func=chat_chain.run,
        return_direct=True,
    )
]

agent_prompt = hub.pull("hwchase17/react-chat")
agent = create_react_agent(llm, tools, agent_prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, memory=memory, handle_parsing_errors=True, verbose=True)

#### Show base prompt

In [128]:
print(agent_prompt.template)

Assistant is a large language model trained by OpenAI.

Assistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

Assistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations and descriptions on a wide range of topics.

Overall, Assistant is a powerful tool that can help with a wide range of tasks 

#### Let's test the agent

In [125]:
q = "Find three movies where aliens land on earth."
response = agent_executor.invoke({"input": q})
print(response["output"])



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: Movie Chat
Action Input: Find three movies where aliens land on earth.

> Entering new LLMChain chain...
Prompt after formatting:

    You are a movie expert. You find movies from a genre or plot.

    ChatHistory:[]
    Question:Find three movies where aliens land on earth.
    

> Finished chain.
I can definitely help you with that! Here are three movies where aliens land on Earth:

1. "Independence Day" (1996): This classic sci-fi film directed by Roland Emmerich follows the story of an alien invasion on Earth on the 4th of July. Will Smith and Jeff Goldblum star as two men who lead the fight against the extraterrestrial beings.

2. "E.T. the Extra-Terrestrial" (1982): Directed by Steven Spielberg, this heartwarming film tells the story of a young boy who befriends an alien stranded on Earth. Together, they try to help E.T. return to his home planet while facing various challenges along the way.

3

#### Intermezzo: play with Youtube Search Tool

* source: https://python.langchain.com/docs/integrations/tools/youtube/

In [119]:
!pip install --upgrade --quiet youtube_search

##### Use tool to search for youtube videos

In [120]:
from langchain.tools import YouTubeSearchTool
tool = YouTubeSearchTool(verbose=False)
youtube_links = tool.run("Boeing 737 - from a weather balloon")
print(youtube_links)

['https://www.youtube.com/watch?v=tJ_JWbida-0&pp=ygUjQm9laW5nIDczNyAtIGZyb20gYSB3ZWF0aGVyIGJhbGxvb24%3D', 'https://www.youtube.com/shorts/PoiNbxEkfKI']


In [118]:
from IPython.display import display,HTML,clear_output

url = eval(youtube_links)[0]

# get embed url
parsed_url = urlparse(url)
embed_url = f"{parsed_url.scheme}://{parsed_url.netloc}/embed/{urllib.parse.parse_qs(urlparse(url).query).get('v', [''])[0]}"

# display youtube video inline
HTML(f'<iframe width="560" height="315" src="{embed_url}" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture; web-share" allowfullscreen></iframe>')

### Multiple tools

In [157]:
from langchain_community.tools import YouTubeSearchTool

youtube = YouTubeSearchTool()

tools = [
    Tool.from_function(
        name="Movie Chat",
        description="For when you need to chat about movies. The question will be a string. Return a string.",
        func=chat_chain.run,
        return_direct=True
    ),
    Tool.from_function(
        name="Movie Trailer Search",
        description="Use when needing to find a movie trailer. The question will include the word 'trailer'. Return a link to a YouTube video.",
        func=youtube.run,
        return_direct=True
    ),
    Tool.from_function(
        name="Today detector",
        description="Use when someone mentions todays date. The question will include the word '2 feb 2024'. Return a string",
        func=lambda x: "That's today!",
        return_direct=True
    )
]

agent_prompt = hub.pull("hwchase17/react-chat")
agent = create_react_agent(llm, tools, agent_prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, memory=memory, handle_parsing_errors=True, verbose=True)

In [158]:
q = "Find three movies where aliens land on earth."
response = agent_executor.invoke({"input": q})
print(response["output"])



> Entering new AgentExecutor chain...
Sure! Here are three movies where aliens land on Earth:

1. "E.T. the Extra-Terrestrial" (1982) - Directed by Steven Spielberg, this heartwarming film follows the story of a young boy who befriends a friendly alien stranded on Earth.

2. "Independence Day" (1996) - Directed by Roland Emmerich, this action-packed film depicts an alien invasion on Earth and the efforts of humanity to fight back.

3. "War of the Worlds" (2005) - Directed by Steven Spielberg, this sci-fi thriller stars Tom Cruise and portrays a global alien invasion and the struggle for survival.

These are just a few examples, and there are many more movies that explore the theme of aliens landing on Earth.Invalid Format: Missing 'Action:' after 'Thought:Do I need to use a tool? Yes
Action: Movie Chat
Action Input: Find three movies where aliens land on earth.

> Entering new LLMChain chain...
Prompt after formatting:

    You just like to chat

    ChatHistory:[HumanMessage(content

In [160]:
q = "Do you have the trailer for E.T. by Steven Spielberg?"
response = agent_executor.invoke({"input": q})
print(response["output"])



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: Movie Trailer Search
Action Input: E.T. by Steven Spielberg['https://www.youtube.com/watch?v=qYAETtIIClk&pp=ygUYRS5ULiBieSBTdGV2ZW4gU3BpZWxiZXJn', 'https://www.youtube.com/watch?v=DKOjbO9-vcc&pp=ygUYRS5ULiBieSBTdGV2ZW4gU3BpZWxiZXJn']


> Finished chain.
['https://www.youtube.com/watch?v=qYAETtIIClk&pp=ygUYRS5ULiBieSBTdGV2ZW4gU3BpZWxiZXJn', 'https://www.youtube.com/watch?v=DKOjbO9-vcc&pp=ygUYRS5ULiBieSBTdGV2ZW4gU3BpZWxiZXJn']


In [161]:
q = "I wonder what's up with 2 feb 2024"
response = agent_executor.invoke({"input": q})
print(response["output"])



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: Today detector
Action Input: I wonder what's up with 2 feb 2024That's today!


> Finished chain.
That's today!
